In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import psycopg2
import plotly.express as px

# Fast to .csv
def csv(df, name):
    df.to_csv(f'{name}.csv',index=False)

# Fast to .xlsx
def excel(df, name):
    df.to_excel(f'{name}.xlsx',index=False)

In [7]:
def authorization():
    login_name = ''
    login_password = ''
    IP = ''
    engine_str = f'mssql+pymssql://{login_name}:{login_password}@{IP}:1433/monitoring'
    engine = create_engine(engine_str)
    return engine

In [8]:
def execute_data_bd():
    query =f'''SELECT
	YEAR(GMT) AS year,
	MONTH(GMT) AS month,
	DATEPART(HOUR, DATEADD(HOUR, 8, GMT)) AS hour,
	HEADING,
	lat,
	lon,
	AVG(SPEED) AS average_speed
FROM
	monitoring.dbo.transport
WHERE
	GMT > '2021-09-01T00:00:00'	AND GMT < '2022-02-01T00:00:00'
	AND DATEPART(HOUR, DATEADD(HOUR, 8, GMT)) >= 5
	AND SPEED >= 0
	AND SPEED <= 61
GROUP BY
	YEAR(GMT),
	MONTH(GMT),
	DATEPART(HOUR, DATEADD(HOUR, 8, GMT)),
	HEADING,
	lat,
	lon
ORDER BY 
	YEAR(GMT),
	MONTH(gmt),
	DATEPART(HOUR, DATEADD(HOUR, 8, GMT))
    '''
    filtered_lat_lon = pd.read_sql_query(query,authorization())
    return filtered_lat_lon

In [9]:
%%time
df = execute_data_bd()

CPU times: user 3min 47s, sys: 1min 20s, total: 5min 7s
Wall time: 49min 36s


In [10]:
df

,year,month,hour,HEADING,lat,lon,average_speed
0,2021,9,5,0.0,52.256725,104.364197,14.0
1,2021,9,5,0.0,52.271416,104.327194,0.0
2,2021,9,5,0.0,52.271530,104.327995,0.0
3,2021,9,5,0.0,52.271538,104.328003,0.0
4,2021,9,5,0.0,52.271725,104.325348,0.0
...,...,...,...,...,...,...,...
16479194,2022,1,23,359.0,52.272530,104.327370,0.0
16479195,2022,1,23,359.0,52.276043,104.257591,17.0
16479196,2022,1,23,359.0,52.281219,104.295845,10.0
16479197,2022,1,23,359.0,52.281395,104.295982,14.0


In [11]:
df_pg = df.copy()

In [38]:
df_pg.month.unique()

array([ 1,  2, 12])

In [43]:
df_pg = df_pg[df_pg.month == 1]
df_pg.month.unique()

array([1])

In [44]:
df_pg.shape

(343656, 4)

In [14]:
df2021 = df_pg[df_pg.year == 2021]

In [12]:
df2022 = df_pg[df_pg.year == 2022]

In [13]:
df2022

,year,month,hour,HEADING,lat,lon,average_speed
13394704,2022,1,5,0.0,52.256699,104.364227,15.0
13394705,2022,1,5,0.0,52.270916,104.325447,0.0
13394706,2022,1,5,0.0,52.271355,104.326111,0.0
13394707,2022,1,5,0.0,52.271431,104.327698,0.0
13394708,2022,1,5,0.0,52.271519,104.328651,0.0
...,...,...,...,...,...,...,...
16479194,2022,1,23,359.0,52.272530,104.327370,0.0
16479195,2022,1,23,359.0,52.276043,104.257591,17.0
16479196,2022,1,23,359.0,52.281219,104.295845,10.0
16479197,2022,1,23,359.0,52.281395,104.295982,14.0


In [15]:
df2021

,year,month,hour,HEADING,lat,lon,average_speed
0,2021,9,5,0.0,52.256725,104.364197,14.0
1,2021,9,5,0.0,52.271416,104.327194,0.0
2,2021,9,5,0.0,52.271530,104.327995,0.0
3,2021,9,5,0.0,52.271538,104.328003,0.0
4,2021,9,5,0.0,52.271725,104.325348,0.0
...,...,...,...,...,...,...,...
13394699,2021,12,23,359.0,52.282284,104.259422,18.0
13394700,2021,12,23,359.0,52.282448,104.297234,15.0
13394701,2021,12,23,359.0,52.286915,104.257812,14.0
13394702,2021,12,23,359.0,52.287010,104.257645,23.0


In [45]:
df_pg['year'] = 2022
df_pg.head(5)

,month,lat,lon,average_speed,year
0,1,52.250294,104.313110,35.0,2022
1,1,52.250717,104.321304,21.2,2022
2,1,52.253651,104.295242,0.0,2022
3,1,52.255116,104.298904,38.0,2022
4,1,52.256207,104.363403,0.0,2022


In [16]:
def authorization_pg():
    login_name = ''
    login_password = ''
    IP = ''
    # engine_str = f'mssql+pymssql://{login_name}:{login_password}@{IP}:1433/monitoring'
    engine_str = f'postgresql://{login_name}:{login_password}@{IP}:5432/postgres'
    engine = create_engine(engine_str)
    return engine

In [17]:
 def generate():
    d = {
        'year':df2021.year
        ,'month':df2021.month
        ,'hour':df2021.hour
        ,'heading':df2021.HEADING
        ,'lat':df2021.lat
        ,'lon':df2021.lon
        ,'average_speed':df2021.average_speed
    }
    df_into = pd.DataFrame(data=d)
    return df_into

In [18]:
 def into_pg_table(pg_table_name):
    dataframe = generate()
    connector = authorization_pg()
    dataframe.to_sql(
        name = pg_table_name,
        con = connector,
        chunksize = 10000,
        index = False,
        if_exists = 'append'
    )

In [19]:
%%time
into_pg_table('dbo_average_speed_tramway')

CPU times: user 2min 45s, sys: 3.53 s, total: 2min 49s
Wall time: 4min 29s


In [20]:
%%time
df_pg_test =  pd.read_sql_query('''select * from dbo_average_speed_tramway''', authorization_pg())
df_pg_test

CPU times: user 1min 52s, sys: 2min 13s, total: 4min 6s
Wall time: 6min 5s


,pm_id,year,month,hour,heading,lat,lon,average_speed
0,5190496,2022,1,17,303.0,52.285084,104.274559,17.0
1,5190497,2022,1,17,303.0,52.285099,104.306030,16.0
2,5190498,2022,1,17,303.0,52.285126,104.274475,20.0
3,5190499,2022,1,17,303.0,52.285126,104.305679,25.0
4,5190500,2022,1,17,303.0,52.285210,104.305267,21.0
...,...,...,...,...,...,...,...,...
16479194,19563690,2021,12,23,359.0,52.282284,104.259422,18.0
16479195,19563691,2021,12,23,359.0,52.282448,104.297234,15.0
16479196,19563692,2021,12,23,359.0,52.286915,104.257812,14.0
16479197,19563693,2021,12,23,359.0,52.287010,104.257645,23.0
